# Libraries

In [1]:
from concurrent.futures import ThreadPoolExecutor
import json
import multiprocessing
import os
import pickle
from flask import Flask , request
from elasticsearch import Elasticsearch
from flask_cors import CORS
from queue import Empty
from urllib.parse import urljoin, urlparse
import os
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from ordered_set import OrderedSet
import re
from sklearn.feature_extraction.text import CountVectorizer
import itertools
from string import ascii_lowercase
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from pathlib import Path
from bs4 import BeautifulSoup, Comment
import requests
from scipy import sparse
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time


/var/folders/j7/hhs6jm2j6mv0cdm7n2z33sx80000gp/T/ipykernel_1422/1768640997.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Data preparasion

## Web crawler

In [2]:
class MultiThreadCrawler:
    def __init__(self, base_url, depth):
        self.base_url = base_url
        extracted_url = urlparse(base_url)
        parent = extracted_url.path[:extracted_url.path.rfind("/") +  1]
        self.root_url = '{}://{}{}'.format(extracted_url.scheme, extracted_url.netloc, parent)
        self.pool = ThreadPoolExecutor(max_workers= multiprocessing.cpu_count() - 1)
        self.to_crawl = multiprocessing.Queue()
        self.to_crawl.put({self.base_url: depth})
        self.stored_folder = Path(os.path.abspath('')).parent / 'Assignment/crawled'

        if not Path(self.stored_folder).exists(): 
            Path.mkdir(self.stored_folder)

        if Path(self.stored_folder / 'url_list.pickle').exists():
            with open(self.stored_folder / 'url_list.pickle', 'rb') as f:
                self.crawled_pages = pickle.load(f)
            print(self.crawled_pages)
        else:
            self.crawled_pages = set([])
    
    def extract_page(self, obj):
        if obj.result():
            result, url, depth = obj.result()
            if result and result.status_code == 200 :
                url_lists = self.parse_links(result.text, depth)
                self.parse_contents(url, result.text, url_lists)

    def get_page(self, url, depth): 
        try:
            res = requests.get(url, timeout=(3,30))
            return res, url, depth
        except requests.RequestException:
            return
    
    def parse_links(self, html, depth):
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', href=True)
        url_lists = []
        for link in links:
            url = link['href']
            url = urljoin(self.root_url, url)
            if not url.endswith("/"):
                url += "/"
            if depth >= 0 and '..' not in url and url not in self.crawled_pages:
                print("Adding {}".format(url))
                self.to_crawl.put({url: depth})
            url_lists.append(url)
        return url_lists
    
    def parse_contents(self, url, html, url_lists):
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
        
        try: 
            soup = BeautifulSoup(html, 'html.parser')
            texts = soup.findAll(string=True)
            visible_texts = filter(tag_visible, texts)
            title = soup.find('title').string.strip()
            text = u" ".join(t.strip() for t in visible_texts).strip()  

            with open(self.stored_folder / (str(hash(url)) + '.txt'), 'w', encoding='utf-8') as f: 
                json.dump({
                    'url': url,
                    'title': title,
                    'text': text,
                    'url_lists': url_lists
                }, f, ensure_ascii=False)
        except:
            pass

    
    def run_scraper(self):
        while True:
            try:
                target = self.to_crawl.get(timeout=10)
                url, depth = [(k, target[k]) for k in target][0]
                if url not in self.crawled_pages:
                    self.crawled_pages.add(url)
                    job = self.pool.submit(self.get_page, url , depth - 1)
                    job.add_done_callback(self.extract_page)
            except Empty:
                with open(self.stored_folder / 'url_list.pickle', 'wb') as f :
                    pickle.dump(self.crawled_pages, f , pickle.HIGHEST_PROTOCOL)
                with open(self.stored_folder/ 'url_list.pickle', 'rb') as f :
                    print(pickle.load(f))
                break
            except Exception as e :
                print(e)
                continue


In [3]:
if __name__ == '__main__':
    s = MultiThreadCrawler("https://camt.cmu.ac.th/index.php/en/", 2)
    s.run_scraper()

{'https://camt.cmu.ac.th/index.php/en/edit_entry.php?area=1&room=1&hour=18&minute=0&year=2024&month=2&day=25/', 'https://camt.cmu.ac.th/images/gallery_in_article/2024022101/DSC03516.jpg/', 'https://camt.cmu.ac.th/images/gallery_in_article/2024022203/DSC03986.jpg/', 'https://camt.cmu.ac.th/images/file_oit_2566/2403.pdf/', 'https://camt.cmu.ac.th/images/gallery_in_article/2024022202/DSC03748.jpg/', 'https://camt.cmu.ac.th/images/gallery_in_article/2024022102/DSC03704.jpg/', 'https://camt.cmu.ac.th/images/gallery_in_article/2024022304/DSC04211.jpg/', 'https://camt.cmu.ac.th/images/gallery_in_article/2024022305/DSC04463.jpg/', 'https://camt.cmu.ac.th/images/gallery_in_article/2024021401/LINE_ALBUM_UMDF_๒๔๐๒๑๔_32.jpg/', 'https://camt.cmu.ac.th/index.php/th/หัวข้อกลุ่มข่าวทั้งหมด/23-ข่าวกิจกรรมนักศึกษา.html/', 'https://camt.cmu.ac.th/images/file_oit_2566/1603.docx/', 'https://camt.cmu.ac.th/index.php/th/major/graduate/graduate-se/', 'https://camt.cmu.ac.th/', 'https://camt.cmu.ac.th/images/g

## Indexer

Customised preprocessor

In [4]:
def custom_preprocessor(s):
    ps = PorterStemmer()
    s = re.sub(r'[^A-Za-z]', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    s = word_tokenize(s)
    s = [word for word in s if len(word)>2]
    s = [ps.stem(w) for w in s]
    s = ' '.join(s)
    return s

Customised BM25

In [5]:
class BM25(object):
    def __init__(self, vectorizer, b=0.75, k1=1.6):
        self.vectorizer = vectorizer
        self.b = b
        self.k1 = k1
    def fit(self , x):
        self.vectorizer.fit(x)
        self.y = super(TfidfVectorizer, self.vectorizer).transform(x)
        self.avdl = self.y.sum(1).mean()
    def transform(self , q):
        b, k1, avdl = self.b , self.k1, self.avdl
    
        len_y = self.y.sum(1).A1
        q, = super(TfidfVectorizer, self.vectorizer).transform([q])
        assert sparse.isspmatrix_csr(q)

        y = self.y.tocsc()[:, q.indices]
        denom = y + (k1 * (1 - b + b * len_y / avdl))[:, None]
        idf = self.vectorizer._tfidf.idf_[None, q.indices] - 1
        numer = y.multiply(np.broadcast_to(idf, y.shape)) * (k1+1)
        return (numer / denom).sum(1).A1

Page rank


In [6]:
class Pr:
    def __init__(self, alpha):
        self.crawled_folder = Path(os.path.abspath('')).parent / 'Assignment/crawled'
        self.alpha = alpha

    def url_extractor(self):
        url_maps = {}
        all_urls = set([])

        for file in os.listdir(self.crawled_folder):
            if file.endswith(".txt"):
                j = json.load(open(os.path.join(self.crawled_folder, file)))
                all_urls.add(j['url'])
                for s in j['url_lists']:
                    all_urls.add(s)
                url_maps[j['url']] = list(set(j['url_lists']))

        all_urls = list(all_urls)
        return url_maps, all_urls
    
    def pr_calc(self):
        url_maps, all_urls = self.url_extractor()
        url_matrix = pd.DataFrame(columns=all_urls, index=all_urls)

        for url in url_maps:
            if len(url_maps[url]) > 0 and len(all_urls) > 0:
                url_matrix.loc[url] = (1 - self.alpha) * (1 / len(all_urls))
                url_matrix.loc[url, url_maps[url]] += self.alpha * (1 / len(url_maps[url]))

        url_matrix.loc[url_matrix.isnull().all(axis=1), :] = (1 / len(all_urls))

        x0 = np.matrix([1 / len(all_urls)] * len(all_urls))
        P = np.asmatrix(url_matrix.values)

        prev_Px = x0
        Px = x0 * P
        i = 0
        while any(abs(np.asarray(prev_Px).flatten() - np.asarray(Px).flatten()) > 1e-8):
            i += 1
            prev_Px = Px
            Px = Px * P

        print('Converged in {0} iterations: {1}'.format(i, np.around(np.asarray(Px).flatten().astype(float), 5)))

        self.pr_result = pd.DataFrame(Px, columns=url_matrix.index, index=['score']).T.loc[list(url_maps.keys())]

In [7]:
class Indexer:
    def __init__(self):
        self.crawled_folder = Path(os.path.abspath('')).parent / 'Assignment/crawled/'
        self.stored_file = 'src/resource/manual_indexer.pkl'
            
        if os.path.isfile(self.stored_file):
            with open(self.stored_file, 'rb') as f:
                cached_dict = pickle.load(f)
            self.__dict__.update(cached_dict)
        else:
            self.run_indexer()
            
    def run_indexer(self):
        self.pr = Pr(alpha=0.85)
        self.pr.pr_calc()
        documents = []
        for file in os.listdir(self.crawled_folder):
            if file.endswith(".txt"):
                try:
                    j = json.load(open(os.path.join(self.crawled_folder, file)))
                    j['id'] = j['url']
                    j['pagerank'] = self.pr.pr_result.loc[j['id']].score
                    print(j['id'] , j['pagerank'] )
                    documents.append(j)
                except:
                    continue
        self.documents = pd.DataFrame.from_dict(documents)

        tfidf_vecotorizor = TfidfVectorizer(preprocessor=custom_preprocessor, stop_words=stopwords.words('english'), use_idf=True)
        self.bm25 = BM25(tfidf_vecotorizor)
        self.bm25.fit(self.documents.apply(lambda s: ' '.join(s[['title', 'text']]), axis=1))
        with open(self.stored_file, 'wb') as f:
            pickle.dump(self.__dict__, f)

    def search(self, query):
        score = self.bm25.transform(query)
        score = pd.DataFrame(score , columns=["score"])
        df = self.documents.join(score)
        df = df.sort_values(by=["score", "pagerank"], ascending=False)
        return df

### Elastic search + PageRank + BM25

In [8]:
app = Flask(__name__)
app.es_client = Elasticsearch(
            "https://localhost:9200",
            basic_auth=("elastic", "gKzjmdkd=QKeITShRujT"),
            ca_certs="../http_ca.crt"
        )
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

app.indexer = Indexer()
app.indexer.run_indexer()

@app.route('/search', methods=['GET'])
def search_es():
    start = time.time()
    response_object = {'status': 'success'}
    argList = request.args.to_dict(flat=False)
    query_term = argList['query'][0]
    results = app.es_client.search(index='simple', source_excludes=['url_lists'], size=100, query={"match": { "text": query_term}})
    end = time.time()
    total_hit = results['hits']['total']['value']
    results_df = pd.DataFrame([[hit["_source"]['title'], hit["_source"]['url'], hit["_source"]['text'][:100], hit["_score"]] for hit in results['hits']['hits']], columns=['title', 'url', 'text', 'score'])
    
    response_object['total_hit'] = total_hit
    response_object['results'] = results_df.to_dict('records')
    response_object['elapse'] = end - start

    return response_object

@app.route('/manual_index', methods=["GET"])
def manual_index():
    
    start = time.time()
    res = {'status': 'success'}
    argList = request.args.to_dict(flat=False)
    query_term = argList['query'][0]
    results = app.indexer.search(query_term)
    end = time.time()
    total_hit = len(results)
    
    res['total_hit'] = total_hit
    res['results'] = results.sort_values("score", ascending=False).drop("url_lists", axis=1).head(100).to_dict('records')
    res['elapse'] = end-start

    return res

Converged in 18 iterations: [0.00086 0.00033 0.00028 ... 0.00027 0.00028 0.0003 ]
https://camt.cmu.ac.th/index.php/en/#philosophy/ 0.0003671860483272728
https://camt.cmu.ac.th/index.php/th/หัวข้อกลุ่มข่าวทั้งหมด/24-ข่าวทั่วไป.html/ 0.0003947386465337889
https://camt.cmu.ac.th/index.php/th/ระเบียบ/พัสดุ/พรบ.html/ 0.0007043409412095834
https://service.camt.cmu.ac.th/onestop/pages/skill_th/130/ 0.0002974779940835539
https://situs-toto.sirekap.com/ 0.0003564796003684603
https://admission.grad.cmu.ac.th/admissions/indexth.php?p=107&id=10577 / 0.0003789546485068808
https://camt.cmu.ac.th/index.php/th/ข้อมูลจัดซื้อจัดจ้าง.html/ 0.0007043409412095834
https://camt.cmu.ac.th/index.php/en/all-news-groups/24-ข่าวทั่วไป.html/ 0.0005903698495154403
https://camt.cmu.ac.th/index.php/en/all-news-groups/18-ข่าวสมัครงาน.html/ 0.0003043555583035173
https://camt.cmu.ac.th/index.php/en/?p=news_detail&key=4bb50e3eab4f8a1fc7ef2827efa99a7f/ 0.0003117463018494916
https://camt.cmu.ac.th/index.php/th/component/us

/Users/CAMT/anaconda3/envs/SE-266e/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(


In [9]:
app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Feb/2024 23:30:20] "GET /manual_index?query=camt HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:30:41] "GET /manual_index?query=camt HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:33:00] "GET /manual_index?query=head HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:33:19] "GET /manual_index?query=game HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:33:23] "GET /manual_index?query= HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:33:25] "GET /manual_index?query= HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:33:27] "GET /manual_index?query= HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:37:28] "GET /manual_index?query=แฟทะ HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:37:54] "GET /manual_index?query=game HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:40:38] "GET /manual_index?query=software HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:42:39] "GET /manual_index?query=games HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2024 23:42:43] 